In [1]:
import sys
sys.path.append('../../src')

from utils import project_root
projroot = project_root()
import warnings
warnings.filterwarnings("ignore")

In [2]:
from models import CCNN
import yaml
from omegaconf import OmegaConf
from matplotlib import pyplot as plt
from utils import num_params

# Load the configuration
file_path = "config.yaml"
with open(file_path, 'r') as file:
    config = OmegaConf.create(yaml.safe_load(file))

name = "test1_ccnn"
torch_model = CCNN(name, in_channels=1, out_channels=2, config=config)
torch_model.load(name)
torch_input = torch_model.example_input[0]
torch_out = torch_model(torch_input)
print(torch_out)

MODEL LOADED!
tensor([[0.7794, 0.8375]], grad_fn=<AddmmBackward0>)


In [3]:
# from core import ExecutorchModel
from edge import ONNXModel
import onnx
import torch

torch_model.eval()

torch.onnx.export(
    torch_model,
    torch_input,
    f"{name}.onnx",
    verbose=True,
    input_names=["input"],
    output_names=["output"],
)

onnx_model = onnx.load(f"{name}.onnx")
onnx.checker.check_model(onnx_model)

Torch IR graph at exception: graph(%input.1 : Float(1, 1, 513, strides=[513, 513, 1], requires_grad=0, device=cpu),
      %backbone.0.conv1.bias : Float(2, strides=[1], requires_grad=1, device=cpu),
      %backbone.0.conv1.Kernel.kernel_net.0.bias : Float(32, strides=[1], requires_grad=1, device=cpu),
      %backbone.0.conv1.Kernel.kernel_net.0.parametrizations.weight.original0 : Float(32, 1, 1, strides=[1, 1, 1], requires_grad=1, device=cpu),
      %backbone.0.conv1.Kernel.kernel_net.0.parametrizations.weight.original1 : Float(32, 1, 1, strides=[1, 1, 1], requires_grad=1, device=cpu),
      %backbone.0.conv1.Kernel.kernel_net.3.bias : Float(32, strides=[1], requires_grad=1, device=cpu),
      %backbone.0.conv1.Kernel.kernel_net.3.parametrizations.weight.original0 : Float(32, 1, 1, strides=[1, 1, 1], requires_grad=1, device=cpu),
      %backbone.0.conv1.Kernel.kernel_net.3.parametrizations.weight.original1 : Float(32, 32, 1, strides=[32, 1, 1], requires_grad=1, device=cpu),
      %back

UnsupportedOperatorError: Exporting the operator 'aten::fft_rfft' to ONNX opset version 17 is not supported. Please feel free to request support or submit a pull request on PyTorch GitHub: https://github.com/pytorch/pytorch/issues.

In [50]:
import numpy as np
import onnxruntime

ort_session = onnxruntime.InferenceSession(f"{name}.onnx", providers=["CPUExecutionProvider"])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(torch_input)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!


In [51]:
import time
import numpy as np

onnx_elapsed = 0
iterations = 10000
onnx_elapsed = []

for i in range(iterations):
    torch_input = torch.rand(torch_model.example_input[0].shape)
    ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(torch_input)}
    start = time.time()
    ort_session.run(None, ort_inputs)
    end = time.time()
    onnx_elapsed.append(end - start)

print(np.mean(onnx_elapsed) * 1000)

0.11729550361633301
